In [1]:
import json
import pandas as pd
from pathlib import Path

In [2]:
train_annfile = "coco/annotations/captions_train2014.json"
train_img_folder = "coco/images/train2014/"

val_annfile = "coco/annotations/captions_val2014.json"
val_img_folder = "coco/images/val2014/"

with open(val_annfile, "r") as infile:
    val = json.load(infile)
    
with open(train_annfile, "r") as infile:
    train = json.load(infile)

In [3]:
train_images = pd.DataFrame(train["images"])[["id", "file_name", "height", "width"]].rename(columns={"id":"image_id"})
train_annotations = pd.DataFrame(train["annotations"])
train_annotations = train_annotations.groupby(by="image_id")["caption"].apply(list).reset_index()

train_df = train_annotations.merge(train_images, on="image_id", how="inner")
train_df["file_path"] = train_df["file_name"].apply(lambda x: Path(train_img_folder+x))
train_df.drop(columns="file_name", inplace=True)


val_images = pd.DataFrame(val["images"])[["id", "file_name", "height", "width"]].rename(columns={"id":"image_id"})
val_annotations = pd.DataFrame(val["annotations"])
val_annotations = val_annotations.groupby(by="image_id")["caption"].apply(list).reset_index()

val_df = val_annotations.merge(val_images, on='image_id', how='inner')
val_df["file_path"] = val_df["file_name"].apply(lambda x: Path(val_img_folder+x))
val_df.drop(columns="file_name", inplace=True)

df = pd.concat([train_df, val_df]).sort_values(by="image_id").reset_index(drop=True)

In [4]:
df

,image_id,caption,height,width,file_path
0,9,[Closeup of bins of food that include broccoli...,480,640,coco/images/train2014/COCO_train2014_000000000...
1,25,[A giraffe eating food from the top of the tre...,426,640,coco/images/train2014/COCO_train2014_000000000...
2,30,"[A flower vase is sitting on a porch stand., W...",428,640,coco/images/train2014/COCO_train2014_000000000...
3,34,[A zebra grazing on lush green grass in a fiel...,425,640,coco/images/train2014/COCO_train2014_000000000...
4,36,[Woman in swim suit holding parasol on sunny d...,640,481,coco/images/train2014/COCO_train2014_000000000...
...,...,...,...,...,...
123282,581906,"[A damaged, leather suit case sitting on a dir...",428,640,coco/images/train2014/COCO_train2014_000000581...
123283,581909,[A boat on a trailer with other boats near it ...,426,640,coco/images/train2014/COCO_train2014_000000581...
123284,581913,[Four donuts in a box with a variety of frosti...,375,500,coco/images/val2014/COCO_val2014_000000581913.jpg
123285,581921,[THERE IS A MAN THAT IS JUMPING HIGH ON A SNOW...,427,640,coco/images/train2014/COCO_train2014_000000581...


In [5]:
# df.to_json("coco/dataset_info.json", orient="records")
df.to_pickle("coco/dataset_info.pkl")